In [ ]:
from typing import Dict, Optional, Union
from pydantic import BaseModel, Field
from datetime import datetime

class CompanyInfo(BaseModel):
    sector: str
    ranking: float
    ticker: str
    refinitive_id: str

class RelationDB:
    def __init__(
        self,
        year: int,
        quarter: str,
        top_n: Optional[int] = 5,
        use_cache: Optional[bool] = False,
    ):
        self.d = gsam_fe_get_top_customers_info_for_companies(
            year, quarter, top_n=top_n, use_cache=use_cache
        )
        self._validate_data()
        
    def _validate_data(self) -> None:
        required_maps = [
            "ticker to bbg cid map",
            "bbg cid to refinitiv id map",
            "top customers final struct",
            "top suppliers final struct"
        ]
        missing = [k for k in required_maps if k not in self.d]
        if missing:
            raise ValueError(f"Missing required maps: {missing}")

    def ticker_to_refinitive_id(self, ticker: str) -> Optional[str]:
        try:
            bbg_cid = self.d["ticker to bbg cid map"].get(ticker)
            if not bbg_cid:
                return None
            return self.d["bbg cid to refinitiv id map"].get(bbg_cid)
        except Exception as e:
            print(f"Error converting ticker {ticker}: {e}")
            return None

    def ticker_to_bbg_cid(self, ticker: str) -> Optional[str]:
        return self.d["ticker to bbg cid map"].get(ticker)

    def bbg_cid_to_ticker(self, bbg_cid: str) -> Optional[str]:
        bbg_cid_to_ticker = {
            y: x for x, y in self.d["ticker to bbg cid map"].items()
        }
        return bbg_cid_to_ticker.get(bbg_cid)

    def get_customers_by_ticker(self, ticker: str) -> Dict[str, CompanyInfo]:
        try:
            bbg_cid = self.ticker_to_bbg_cid(ticker)
            if not bbg_cid:
                return {}
                
            result = {}
            customer_data = self.d["top customers final struct"].get(bbg_cid, {})
            
            for k, v in customer_data.items():
                customer_ticker = self.bbg_cid_to_ticker(k)
                if not customer_ticker:
                    continue
                    
                refinitive_id = self.ticker_to_refinitive_id(customer_ticker)
                if not refinitive_id:
                    continue
                
                v["ticker"] = customer_ticker
                v["refinitive_id"] = refinitive_id
                result[k] = CompanyInfo(**v)
                
            return result
        except Exception as e:
            print(f"Error getting customers for ticker {ticker}: {e}")
            return {}

    def get_suppliers_by_ticker(self, ticker: str) -> Dict[str, CompanyInfo]:
        try:
            bbg_cid = self.ticker_to_bbg_cid(ticker)
            if not bbg_cid:
                return {}
                
            result = {}
            supplier_data = self.d["top suppliers final struct"].get(bbg_cid, {})
            
            for k, v in supplier_data.items():
                supplier_ticker = self.bbg_cid_to_ticker(k)
                if not supplier_ticker:
                    continue
                    
                refinitive_id = self.ticker_to_refinitive_id(supplier_ticker)
                if not refinitive_id:
                    continue
                
                v["ticker"] = supplier_ticker
                v["refinitive_id"] = refinitive_id
                result[k] = CompanyInfo(**v)
                
            return result
        except Exception as e:
            print(f"Error getting suppliers for ticker {ticker}: {e}")
            return {}